# Introduction
In this project, I want to make a data exploration & query using SQLite  then data visualization using Tableau


### **Disclaimer**
I'm doing this project inspired by a project from Alex the Analyst: https://www.youtube.com/playlist?list=PLUaB-1hjhk8H48Pj32z4GZgGWyylqv85f


### **Dataset**
I get the dataset from:  https://ourworldindata.org/covid-deaths

From that dataset, we break it down into 2 data, covid_death and covid_vaccinations



In [ ]:
# import libraries

import numpy as np 
import pandas as pd 
import sqlite3

In [ ]:
print("SQLite Version is:", sqlite3.sqlite_version)
print("DB-API Version is:", sqlite3.version)

SQLite Version is: 3.22.0
DB-API Version is: 2.6.0


In [ ]:
# define connection and cursor
# conenction is used to connect with database
database = '/content/drive/MyDrive/data_anal_project/Covid-19-2021/covid2021.db'
connection = sqlite3.connect(database)

# cursor is used to interact with database through sql command
cursor = connection.cursor()

In [ ]:
# read the csv files
path1 = '/content/drive/MyDrive/data_anal_project/Covid-19-2021/covid_death.csv'
cov_death = pd.read_csv(path1)

In [ ]:
cov_death.head(2)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,population,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2/24/2020,5.0,5.0,NaN,NaN,NaN,NaN,39835428.0,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/2020,5.0,0.0,NaN,NaN,NaN,NaN,39835428.0,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# read the csv files
path2 = '/content/drive/MyDrive/data_anal_project/Covid-19-2021/covid_vaccinations.csv'
cov_vac = pd.read_csv(path2)

In [ ]:
cov_vac.head(2)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2/24/2020,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/2020,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [ ]:
# import data from excel to sql
cov_death.to_sql('covDeath', connection, if_exists='replace', index=False)
cov_vac.to_sql('covVac', connection, if_exists='replace', index=False)

# Select The data

## cov_death_concise

In [ ]:
# select Data that we are foing to be using 

command1 = '''
        SELECT 
          Location,
          date,
          new_cases,
          total_deaths,
          population
        FROM covDeath
        WHERE
          continent is not null
        ORDER BY 
          Location ASC,
          substr (date,5,8) || substr(date,1,2) || substr(date,3,4) ASC;
        '''
cov_death_concise = pd.read_sql(command1, connection)
cov_death_concise.head(3)

,location,date,new_cases,total_deaths,population
0,Afghanistan,10/1/2020,17.0,1462.0,39835428.0
1,Afghanistan,10/2/2020,5.0,1462.0,39835428.0
2,Afghanistan,10/3/2020,7.0,1466.0,39835428.0


In [ ]:
# Export data to csv
cov_death_concise.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/cov_death_concise.csv', 
                index = False, 
                header=True)

## cov_max_case_infect_population

In [ ]:
# Looking at Country with Highest Infection Rate compared to Population
command4 = '''
        SELECT 
          Location,
          population,
          MAX(total_cases) AS highest_infection,
          MAX((total_cases/population)) * 100 AS population_infected_percent
        FROM covDeath
        WHERE
          continent is not null
        GROUP BY
          Location
        ORDER BY 
          population_infected_percent DESC
        '''
cov_max_case_infect_population = pd.read_sql(command4, connection)
cov_max_case_infect_population.head()


,location,population,highest_infection,population_infected_percent
0,Montenegro,628051.0,140922.0,22.437987
1,Seychelles,98910.0,22102.0,22.345567
2,Andorra,77354.0,15404.0,19.913644
3,Georgia,3979773.0,692240.0,17.393957
4,San Marino,34010.0,5497.0,16.162893


In [ ]:
# Export data to csv
cov_max_case_infect_population.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/cov_max_case_infect_population.csv', 
                index = False, 
                header=True)

## cov_max_case_infect_population_date

In [ ]:
# Looking at Country with Highest Infection Rate compared to Population
command4_1 = '''
        SELECT 
          Location,
          population,
          date,
          MAX(total_cases) AS highest_infection,
          MAX((total_cases/population)) * 100 AS population_infected_percent
        FROM covDeath
        WHERE
          continent is not null
        GROUP BY
          Location,
          Population,
          Date
        ORDER BY 
          population_infected_percent DESC
        '''
cov_max_case_infect_population_date = pd.read_sql(command4_1, connection)
cov_max_case_infect_population_date.head()


,location,population,date,highest_infection,population_infected_percent
0,Montenegro,628051.0,10/24/2021,140922.0,22.437987
1,Montenegro,628051.0,10/23/2021,140489.0,22.369043
2,Seychelles,98910.0,10/22/2021,22102.0,22.345567
3,Seychelles,98910.0,10/23/2021,22102.0,22.345567
4,Seychelles,98910.0,10/24/2021,22102.0,22.345567


In [ ]:
# Export data to csv
cov_max_case_infect_population_date.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/cov_max_case_infect_population_date.csv', 
                index = False, 
                header=True)

## cov_case_death_per_date

In [ ]:
# Global Numbers per date

command7 = '''
        SELECT 
          date,
          SUM(new_cases) as Total_cases,
          SUM(new_deaths) as Total_deaths,
          (Total_deaths/Total_cases)*100 as percentage_death  
        FROM covDeath
        WHERE
          continent is not null 
        GROUP BY
          date
        ORDER BY 
          substr(date,5,8) || substr(date,1,4) ASC;
        ''' 
cov_case_death_per_date = pd.read_sql(command7, connection)
cov_case_death_per_date.head(5)

,date,Total_cases,Total_deaths,percentage_death
0,1/10/2020,NaN,NaN,NaN
1,1/11/2020,NaN,NaN,NaN
2,1/12/2020,NaN,NaN,NaN
3,1/13/2020,NaN,NaN,NaN
4,1/14/2020,NaN,NaN,NaN


In [ ]:
# Export data to csv
cov_case_death_per_date.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/cov_case_death_per_date.csv', 
                index = False, 
                header=True)

## global_numbers_death

In [ ]:
# Global Numbers Total

command7_1 = '''
        SELECT 
          date,
          SUM(new_cases) as Total_cases,
          SUM(new_deaths) as Total_deaths,
          (Total_deaths/Total_cases)*100 as percentage_death  
        FROM covDeath
        WHERE
          continent is not null 
        ORDER BY 
          substr(date,5,8) || substr(date,1,4) ASC;
        ''' 
global_numbers_death = pd.read_sql(command7_1, connection)
global_numbers_death

,date,Total_cases,Total_deaths,percentage_death
0,10/24/2021,243044801.0,4931468.0,3.519762


In [ ]:
# Export data to csv
global_numbers_death.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/global_numbers_death.csv', 
                index = False, 
                header=True)

## sum_death_each_country

In [ ]:
# Number of sum death each country excluding world, europe union and international (these are not country but group of country)

command7_2 = '''
        SELECT 
          location, 
          SUM(new_deaths) AS total_death_count
        FROM covDeath 
        WHERE 
          continent is not null 
          AND
          location not in ('World', 'European Union', 'International')
        GROUP BY location
        ORDER BY total_death_count DESC
        ;
        ''' 

sum_death_each_country = pd.read_sql(command7_2, connection)
sum_death_each_country.head(15)


,location,total_death_count
0,United States,735941.0
1,Brazil,605644.0
2,India,454712.0
3,Mexico,281987.0
4,Russia,226464.0
5,Peru,200052.0
6,Indonesia,143205.0
7,United Kingdom,139950.0
8,Italy,131826.0
9,Colombia,127067.0


In [ ]:
# Export data to csv
sum_death_each_country.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/sum_death_each_country.csv', 
                index = False, 
                header=True)

## total_vac_each_country_per_date

In [ ]:
# Number of sum death each country excluding world, europe union and international (these are not country but group of country)

command14 = '''
        SELECT
          dea.continent, 
          dea.location, 
          dea.date, 
          dea.population, 
          MAX(vac.total_vaccinations) AS RollingPeopleVaccinated 
        FROM covDeath AS dea
        JOIN covVac AS vac
          ON dea.location = vac.location
          AND dea.date = vac.date
        WHERE 
          dea.continent is not null 
        GROUP BY 
          dea.continent,
          dea.location,
          dea.date,
          dea.population
        ORDER BY 1,2,3
        ;
        ''' 

total_vac_each_country_per_date = pd.read_sql(command14, connection)
total_vac_each_country_per_date.tail(15)


,continent,location,date,population,RollingPeopleVaccinated
120246,South America,Venezuela,9/3/2021,28704947.0,NaN
120247,South America,Venezuela,9/30/2020,28704947.0,NaN
120248,South America,Venezuela,9/30/2021,28704947.0,NaN
120249,South America,Venezuela,9/4/2020,28704947.0,NaN
120250,South America,Venezuela,9/4/2021,28704947.0,NaN
120251,South America,Venezuela,9/5/2020,28704947.0,NaN
120252,South America,Venezuela,9/5/2021,28704947.0,NaN
120253,South America,Venezuela,9/6/2020,28704947.0,NaN
120254,South America,Venezuela,9/6/2021,28704947.0,NaN
120255,South America,Venezuela,9/7/2020,28704947.0,NaN


In [ ]:
# Export data to csv
sum_death_each_country.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/total_vac_each_country_per_date.csv', 
                index = False, 
                header=True)

## pop_vac

In [ ]:
# USE CTE
# A Common Table Expression, also called as CTE in short form, 
# is a temporary named result set that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement.

command10_1 = '''
        with popvsVac (continent, location, date, population, new_vaccinations, rolling_people_vacc)
        AS 
        (
        SELECT 
          dea.continent,
          dea.location, 
          dea.date,
          dea.population,
          vac.new_vaccinations,
          vac.total_vaccinations as rolling_people_vacc
        FROM covDeath AS dea
        JOIN covVac AS vac
          ON 
            dea.location = vac.location
            and
            dea.date = vac.date
        )
        SELECT *, (rolling_people_vacc/population)*100 AS percent_vac_sum
        FROM popvsVac
        ;
        ''' 

pop_vac = pd.read_sql(command10_1, connection)
pop_vac.tail(15)


,continent,location,date,population,new_vaccinations,rolling_people_vacc,percent_vac_sum
125950,Africa,Zimbabwe,10/10/2021,15092171.0,9206.0,5581524.0,36.982910
125951,Africa,Zimbabwe,10/11/2021,15092171.0,13284.0,5594808.0,37.070929
125952,Africa,Zimbabwe,10/12/2021,15092171.0,17668.0,5612476.0,37.187996
125953,Africa,Zimbabwe,10/13/2021,15092171.0,20058.0,5632534.0,37.320900
125954,Africa,Zimbabwe,10/14/2021,15092171.0,21733.0,5654267.0,37.464902
125955,Africa,Zimbabwe,10/15/2021,15092171.0,20471.0,5674738.0,37.600541
125956,Africa,Zimbabwe,10/16/2021,15092171.0,13220.0,5687958.0,37.688136
125957,Africa,Zimbabwe,10/17/2021,15092171.0,13717.0,5701675.0,37.779025
125958,Africa,Zimbabwe,10/18/2021,15092171.0,41313.0,5742988.0,38.052763
125959,Africa,Zimbabwe,10/19/2021,15092171.0,13759.0,5756747.0,38.143929


In [ ]:
# Export data to csv
pop_vac.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/pop_vac.csv', 
                index = False, 
                header=True)

## cov_max_death_count_continent

In [ ]:
# Breaking things down by continent
# Looking at at highest death count in each continent

command6 = '''
        SELECT 
          location,
          MAX(CAST(total_deaths as int)) as total_death_count_max
        FROM covDeath
        WHERE
          continent is null
          AND
          location not in ('World', 'European Union', 'International')
        GROUP BY
          location
        ORDER BY 
          total_death_count_max DESC
        ''' 
cov_max_death_count_continent = pd.read_sql(command6, connection)
cov_max_death_count_continent.head(15)


,location,total_death_count_max
0,Europe,1284873
1,South America,1167175
2,Asia,1164166
3,North America,1112565
4,Africa,216931
5,Oceania,2685


In [ ]:
# Export data to csv
cov_max_death_count_continent.to_csv(r'/content/drive/MyDrive/data_anal_project/Covid-19-2021/csv_file_to_tableau/cov_max_death_count_continent.csv', 
                index = False, 
                header=True)

## Another Query

In [ ]:
# Looking at Total Cases vs Total deaths in Indonesia
# Shows likelihood of dying if you contract covid in Indonesia
command2 = '''
        SELECT 
          Location,
          date,
          total_cases,
          total_deaths,
          (total_deaths/total_cases) * 100 AS percentage_death
        FROM covDeath
        WHERE
          location like '%indo%' AND continent is not null
        '''
cov_death_percent = pd.read_sql(command2, connection)
cov_death_percent.head(3)

,location,date,total_cases,total_deaths,percentage_death
0,Indonesia,3/2/2020,2.0,NaN,NaN
1,Indonesia,3/3/2020,2.0,NaN,NaN
2,Indonesia,3/4/2020,2.0,NaN,NaN


In [ ]:
# Looking at Total Cases vs Population in this world
# Show what percentage of population that got covid

command3 = '''
        SELECT 
          Location,
          date,
          population,
          total_cases,
          total_deaths,
          (total_cases/population) * 100 AS population_infected_percent
        FROM covDeath
        Where 
          location like '%indo%' AND continent is not null
        '''
cov_case_percent = pd.read_sql(command3, connection)
cov_case_percent.head(3)

,location,date,population,total_cases,total_deaths,population_infected_percent
0,Indonesia,3/2/2020,276361788.0,2.0,NaN,7.236891e-07
1,Indonesia,3/3/2020,276361788.0,2.0,NaN,7.236891e-07
2,Indonesia,3/4/2020,276361788.0,2.0,NaN,7.236891e-07


In [ ]:
# Looking at highest death count in each countries

command5 = '''
        SELECT 
          Location,
          MAX(CAST(total_deaths as int)) as total_death_count_max
        FROM covDeath
        WHERE
          continent is not null
        GROUP BY
          Location
        ORDER BY 
          total_death_count_max DESC
        '''
cov_max_death_count_country = pd.read_sql(command5, connection)
cov_max_death_count_country.head(10)


,location,total_death_count_max
0,United States,735941.0
1,Brazil,605644.0
2,India,454712.0
3,Mexico,286259.0
4,Russia,226464.0
5,Peru,200052.0
6,Indonesia,143205.0
7,United Kingdom,139950.0
8,Italy,131826.0
9,Colombia,127067.0


In [ ]:
# Looking at total population vs vaccinations

# command8 = '''
#         SELECT 
#           dea.continent,
#           dea.location, 
#           dea.date,
#           dea.population,
#           vac.new_vaccinations
#           SUM(vac.new_vaccinations) OVER (PARTITION BY dea.location ORDER BY dea.Location AND dea.date) AS rolling_people_vacc
#         FROM covDeath as dea
#         JOIN covVac as vac
#           ON 
#             dea.location = vac.location
#             and
#             dea.date = vac.datein
#         WHERE 
#           dea.continent is not null
#         GROUP BY
#           2,3;
#         ''' 


command8 = '''
        SELECT 
          dea.continent,
          dea.location, 
          dea.date,
          dea.population,
          vac.new_vaccinations,
          t1.rolling_people_vacc
        FROM covDeath AS dea
        JOIN covVac AS vac
          ON 
            dea.location = vac.location
            and
            dea.date = vac.date
        JOIN (
          SELECT 
            SUM(new_vaccinations) AS rolling_people_vacc,
            location
          FROM covVac
          GROUP BY location
          ORDER BY location AND date
        ) AS t1 
          ON t1.location = dea.location
        WHERE 
          dea.continent is not null 
        ORDER BY
          2,3;
        ''' 
cov_join_death_vac = pd.read_sql(command8, connection)
cov_join_death_vac.head(15)

,continent,location,date,population,new_vaccinations,rolling_people_vacc
0,Asia,Afghanistan,1/1/2021,39835428.0,NaN,6874.0
1,Asia,Afghanistan,1/10/2021,39835428.0,NaN,6874.0
2,Asia,Afghanistan,1/11/2021,39835428.0,NaN,6874.0
3,Asia,Afghanistan,1/12/2021,39835428.0,NaN,6874.0
4,Asia,Afghanistan,1/13/2021,39835428.0,NaN,6874.0
5,Asia,Afghanistan,1/14/2021,39835428.0,NaN,6874.0
6,Asia,Afghanistan,1/15/2021,39835428.0,NaN,6874.0
7,Asia,Afghanistan,1/16/2021,39835428.0,NaN,6874.0
8,Asia,Afghanistan,1/17/2021,39835428.0,NaN,6874.0
9,Asia,Afghanistan,1/18/2021,39835428.0,NaN,6874.0


In [ ]:
command9 = '''
        SELECT 
          dea.continent,
          dea.location, 
          dea.date,
          dea.population,
          t1.rolling_people_vacc
        FROM covDeath AS dea
        JOIN (
          SELECT 
            SUM(new_vaccinations) AS rolling_people_vacc,
            location
          FROM covVac
          GROUP BY location
          ORDER BY location AND date) t1 
            ON t1.location = dea.location
        WHERE 
          dea.continent is not null
        ORDER BY
          2,3;
        ''' 
ff = pd.read_sql(command9, connection)
ff.head(5)

,continent,location,date,population,rolling_people_vacc
0,Asia,Afghanistan,1/1/2021,39835428.0,6874.0
1,Asia,Afghanistan,1/10/2021,39835428.0,6874.0
2,Asia,Afghanistan,1/11/2021,39835428.0,6874.0
3,Asia,Afghanistan,1/12/2021,39835428.0,6874.0
4,Asia,Afghanistan,1/13/2021,39835428.0,6874.0


In [ ]:
# # Cross Join
# command92 = '''
#         SELECT 
#           vv.continent,
#           vv.location, 
#           vv.date,
#           vv.new_vaccinations,
#           SUM(vac.new_vaccinations) as cumulative
#         FROM covVac as vv, covVac as vac
#         WHERE 
#           vac.new_vaccinations <= vv.new_vaccinations 
#           AND
#           vv.continent is not null
#         ORDER BY
#           2,3;
#         ''' 


# stackoverflow.com/questions/5606560/how-do-i-calculate-a-running-sum-on-a-sqlite-query
command92 = '''
        SELECT 
          location, 
          date,
          new_vaccinations,
          SUM(new_vaccinations) as cumulative
        FROM covVac
        WHERE 
         continent is not null
        ORDER BY
          2,3;
        ''' 
ff = pd.read_sql(command92, connection)
ff.head(5)

,location,date,new_vaccinations
0,Argentina,1/1/2020,NaN
1,Mexico,1/1/2020,NaN
2,Peru,1/1/2020,NaN
3,Afghanistan,1/1/2021,NaN
4,Albania,1/1/2021,NaN


In [ ]:
# USE CTE
# A Common Table Expression, also called as CTE in short form, 
# is a temporary named result set that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement.

command10 = '''
        with popvsVac (continent, location, date, population, new_vaccinations, rolling_people_vacc)
        AS 
        (
        SELECT 
          dea.continent,
          dea.location, 
          dea.date,
          dea.population,
          vac.new_vaccinations,
          t1.rolling_people_vacc
        FROM covDeath AS dea
        JOIN covVac AS vac
          ON 
            dea.location = vac.location
            and
            dea.date = vac.date
        JOIN (
          SELECT 
            SUM(new_vaccinations) AS rolling_people_vacc,
            location
          FROM covVac
          GROUP BY location
          ORDER BY location AND date
        ) AS t1 
          ON t1.location = dea.location
        WHERE 
          dea.continent is not null 
        )
        SELECT *, (rolling_people_vacc/population)*100 AS percent_vac_sum
        FROM popvsVac
        ;
        ''' 

cov_join_death_vac = pd.read_sql(command10, connection)
cov_join_death_vac.head(15)


,continent,location,date,population,new_vaccinations,rolling_people_vacc,percent_vac_sum
0,Asia,Afghanistan,2/24/2020,39835428.0,NaN,6874.0,0.017256
1,Asia,Afghanistan,2/25/2020,39835428.0,NaN,6874.0,0.017256
2,Asia,Afghanistan,2/26/2020,39835428.0,NaN,6874.0,0.017256
3,Asia,Afghanistan,2/27/2020,39835428.0,NaN,6874.0,0.017256
4,Asia,Afghanistan,2/28/2020,39835428.0,NaN,6874.0,0.017256
5,Asia,Afghanistan,2/29/2020,39835428.0,NaN,6874.0,0.017256
6,Asia,Afghanistan,3/1/2020,39835428.0,NaN,6874.0,0.017256
7,Asia,Afghanistan,3/2/2020,39835428.0,NaN,6874.0,0.017256
8,Asia,Afghanistan,3/3/2020,39835428.0,NaN,6874.0,0.017256
9,Asia,Afghanistan,3/4/2020,39835428.0,NaN,6874.0,0.017256


In [ ]:
# # USE TEMP TABLE

# command11 = '''
#         DROP Table if exists #percentPopulationVaccinated
#         CREATE TABLE #percentPopulationVaccinated(
#           continent nvarchar(255),
#           location nvarchar(255),
#           date datetime,
#           population numeric,
#           new_vaccinations numeric,
#           rolling_people_vacc numeric
#         )

#         INSERT INTO #percentPopulationVaccinated
        
#         SELECT 
#           dea.continent,
#           dea.location, 
#           dea.date,
#           dea.population,
#           vac.new_vaccinations,
#           t1.rolling_people_vacc
#         FROM covDeath AS dea
#         JOIN covVac AS vac
#           ON 
#             dea.location = vac.location
#             and
#             dea.date = vac.date
#         JOIN (
#           SELECT 
#             SUM(new_vaccinations) AS rolling_people_vacc,
#             location
#           FROM covVac
#           GROUP BY location
#           ORDER BY location AND date
#         ) AS t1 
#           ON t1.location = dea.location
#         WHERE 
#           dea.continent is not null 
#         )
        
#         SELECT *, (rolling_people_vacc/population)*100 AS percent_vac_sum
#         FROM popvsVac
#         ;
#         ''' 

# cov_join_death_vac_percent = pd.read_sql(command11, connection)
# cov_join_death_vac_percent.head(15)


In [ ]:
# # Creating View to store data for later visualizations

# command12_2 = '''
#         CREATE VIEW PercentPopulationVaccinated_1
#         AS
#         SELECT 
#           dea.continent,
#           dea.location, 
#           dea.date,
#           dea.population,
#           vac.new_vaccinations,
#           t1.rolling_people_vacc
#         FROM covDeath AS dea
#         JOIN covVac AS vac
#           ON 
#             dea.location = vac.location
#             and
#             dea.date = vac.date
#         JOIN (
#           SELECT 
#             SUM(new_vaccinations) AS rolling_people_vacc,
#             location
#           FROM covVac
#           GROUP BY location
#           ORDER BY location AND date
#         ) AS t1 
#           ON t1.location = dea.location
#         WHERE 
#           dea.continent is not null;
#         ''' 

# view_1 = pd.read_sql(command12_2, connection)

# # command12_1 = '''
# #         DROP VIEW if exists PercentPopulationVaccinated;
# #         ''' 

# # pd.read_sql(command12_1, connection)

In [ ]:
# # Creating View to store data for later visualizations

# command13a = '''
#         DROP VIEW IF EXISTS example_1
#         ''' 

# pd.read_sql(command13a, connection)


In [ ]:
# # Creating View to store data for later visualizations

# command13 = '''
#         DROP VIEW IF EXISTS example_1
#         CREATE 
#           VIEW example_1
#         AS
#         SELECT 
#           continent,
#           location, 
#           date,
#           population
#         FROM covDeath
#         WHERE 
#           continent is not null
#         ''' 

# view_2 = pd.read_sql(command13, connection)


In [ ]:

# conn.commit used to confirm any changes you make
connection.commit()

# conn.close used to close the connection you make
connection.close()